In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu indic-transliteration soundfile requests gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import torch
import pickle
from indic_transliteration.sanscript import transliterate, ITRANS, DEVANAGARI
import requests
import soundfile as sf
import numpy as np
import faiss
import json
import os
import gradio as gr
from google.colab import drive

In [ ]:
STT_SERVICE_URL = "https://942f-34-87-3-101.ngrok-free.app/stt"
TTS_SERVICE_URL = "https://942f-34-87-3-101.ngrok-free.app/tts"
OPENROUTER_API_KEY = "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"

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Chatbot/KCCNewHindiDataset.xlsx'
df = pd.read_excel(file_path)
database = {row["Question"]: row["Answer"] for _, row in df.iterrows()}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
bi_encoder = SentenceTransformer('sentence-transformers/LaBSE', device=device)
question_embeddings = bi_encoder.encode(list(database.keys()), convert_to_numpy=True, show_progress_bar=True)

dimension = question_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(question_embeddings)

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

In [ ]:
def normalize_code_mixed(text):
    words = text.split()
    normalized = []
    for word in words:
        if all(ord(c) < 128 for c in word):  # Roman
            try:
                dev_word = transliterate(word, ITRANS, DEVANAGARI)
            except:
                dev_word = word
            normalized.append(dev_word)
        else:
            normalized.append(word)
    return ' '.join(normalized)

In [ ]:
def needs_transliteration(text):
    has_devanagari = any(ord(c) > 127 for c in text)
    has_roman = any(ord(c) < 128 and c.isalpha() for c in text)
    return has_roman and (not has_devanagari or has_devanagari)

In [ ]:
def speech_to_text(audio_file_path):
    try:
        with open(audio_file_path, 'rb') as audio_file:
            files = {'audio': audio_file}
            response = requests.post(STT_SERVICE_URL, files=files)
            if response.status_code == 200:
                return response.json()['text']
            else:
                print(f"STT Error: {response.status_code} - {response.text}")
                return None
    except Exception as e:
        print(f"STT Service Error: {str(e)}")
        return None

In [ ]:
def text_to_speech(text, output_file="response.wav"):
    try:
        response = requests.post(TTS_SERVICE_URL, json={'text': text})

        if response.status_code == 200:
            with open(output_file, 'wb') as f:
                f.write(response.content)
            return output_file
        else:
            print(f"TTS Error: {response.status_code}, {response.text}")
            return None

    except Exception as e:
        print(f"TTS Error: {str(e)}")
        return None


In [ ]:
def get_answer_from_deepseek(question, context=None):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "http://localhost",
        "X-Title": "Chatbot",
    }

    prompt = f"""
    This is the user's query: {question}
    {f"This is the context from our database that might help answer it: {context}" if context else ""}
    Please provide a helpful, conversational response in Hindi (Devanagari script).
    """

    data = json.dumps({
        "model": "deepseek/deepseek-chat-v3-0324:free",
        "messages": [{"role": "user", "content": prompt}]
    })

    try:
        response = requests.post(url, headers=headers, data=data)
        if response.status_code == 200:
            response_data = response.json()
            answer = response_data.get("choices", [{}])[0].get("message", {}).get("content", "No answer received.")
            return answer
        else:
            return f"Error: {response.status_code}, {response.text}"
    except Exception as e:
        return f"API Error: {str(e)}"

In [ ]:
def get_best_answer(query, top_k=5):
    if not query:
        return "कृपया एक मान्य प्रश्न प्रदान करें।"

    if needs_transliteration(query):
        query = normalize_code_mixed(query)

    query_embedding = bi_encoder.encode([query], convert_to_numpy=True)[0]
    scores, indices = faiss_index.search(np.array([query_embedding]), top_k)

    top_qas = [(list(database.keys())[i], list(database.values())[i]) for i in indices[0]]

    cross_inputs = [(query, q) for q, a in top_qas]
    rerank_scores = cross_encoder.predict(cross_inputs)

    best_idx = int(np.argmax(rerank_scores))
    best_score = rerank_scores[best_idx]

    if best_score > 0.7:
        best_match = top_qas[best_idx]
        print(f"डेटाबेस में {best_score:.2f} स्कोर के साथ मिलान मिला")
        print(f"मिलान किया गया प्रश्न: {best_match[0]}")
        print(f"डेटाबेस उत्तर: {best_match[1]}")
        refined_answer = best_match[1]
        refined_answer = get_answer_from_deepseek(query, best_match[1])
        return refined_answer
    else:
        print("डेटाबेस में कोई अच्छा मिलान नहीं मिला, सीधे DeepSeek से पूछताछ की जा रही है")
        return get_answer_from_deepseek(query)


In [ ]:
def chatbot(text_input, audio_input):
    if text_input and audio_input:
        return "कृपया एक बार में केवल एक इनपुट (पाठ या ऑडियो) प्रदान करें।", None
    if not text_input and not audio_input:
        return "कृपया या तो पाठ प्रश्न या ऑडियो फ़ाइल प्रदान करें।", None

    if audio_input:
        print("ऑडियो इनपुट संसाधित किया जा रहा है...")
        query = speech_to_text(audio_input)
        if not query:
            return "क्षमा करें, मैं ऑडियो को समझ नहीं सका।", None
    else:
        query = text_input

    print(f"उपयोगकर्ता प्रश्न: {query}")

    answer = get_best_answer(query)

    audio_file = text_to_speech(answer)

    return answer, audio_file

In [ ]:
iface = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(label="Text Input (optional)", placeholder="Enter your question here..."),
        gr.Audio(label="Audio Input (WAV)", type="filepath"),
    ],
    outputs=[
        gr.Markdown(label="Text Output"),
        gr.Audio(label="Audio Output (WAV)"),
    ],
    title="KrishiBot",
    description="Provide only one input at a time (Text or Audio)."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3ad1965e148f984bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
iface.close()

NameError: name 'iface' is not defined